In [1]:
from pyspark import SparkFiles

url = 'https://github.com/dlhinkley/c772-capstone-project/raw/master/data/assessment_items.csv'

spark.sparkContext.addFile(url)

file = "file://" + SparkFiles.get("assessment_items.csv")

dfRaw = spark.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load(file)

In [2]:
from pyspark import SparkFiles

url = 'https://github.com/dlhinkley/c772-capstone-project/raw/master/data/descriptions.csv'

spark.sparkContext.addFile(url)

file = "file://" + SparkFiles.get("descriptions.csv")

dfDesc = spark.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load(file)

In [3]:
# Save as database view
dfRaw.createOrReplaceTempView("raw_data")
dfRaw.cache()


Out[8]: DataFrame[org_sk: string, section_sk: string, ced_assignment_type_code: string, is_affecting_grade: boolean, assessment_sk: string, assessment_instance_sk: string, number_of_learners: int, number_of_distinct_instance_items: int, assignment_max_attempts: int, assignment_late_submission: boolean, assignment_final_submission_date: string, assignment_start_date: string, assignment_due_date: string, min_student_start_datetime: string, max_student_stop_datetime: string, assignment_attempt_number: int, learner_assignment_attempt_sk: string, was_fully_scored_datetime: string, was_submitted_datetime_actual: string, was_in_progress_datetime: string, is_force_scored: boolean, is_manual_scoring_required: boolean, student_start_datetime: string, student_stop_datetime: string, item_is_offline_scored: boolean, assessment_instance_attempt_sk: string, learner_attempt_status: string, learner_sk: string, points_possible_unweighted: double, final_score_unweighted: int, scored_datetime: string, learner_assigned_item_attempt_sk: string, assessment_item_response_sk: string, item_type_code_name: string, scoring_type_code: string, response_correctness: string, assigned_item_status: string, is_deleted: boolean]

In [4]:
# Describe data
desc = dfRaw.describe()

In [5]:
from pyspark.sql import functions as F
# Save field names
identifierFields = dfDesc.filter("type = 'Categorical Identifier'").select("field")
nominalFields    = dfDesc.filter("type = 'Categorical Nominal'").select("field")
continousFields  = dfDesc.filter("type = 'Numeric Continous'").select("field")
intervalFields   = dfDesc.filter("type = 'Categorical Interval'").select("field")
binaryFields     = dfDesc.filter("type = 'Categorical Binary'").select("field")

In [6]:
#continousFields.show(40,False)
desc.select("summary",
            "assignment_attempt_number",
            "assignment_max_attempts" ,
            "final_score_unweighted",
            "number_of_distinct_instance_items",
            "number_of_learners",
            "points_possible_unweighted" ).show(5,False)

--------------------------------------------------------------------------- 
 AttributeError Traceback (most recent call last)
 <command-2210643164324647> in <module> 
 6 "number_of_distinct_instance_items" , 
 7 "number_of_learners" , 
 ----> 8 "points_possible_unweighted" ).show(5,False).transpose()
 
 AttributeError : 'NoneType' object has no attribute 'transpose'

In [7]:
#nominalFields.show(40,False)
desc.select("summary",
            "assigned_item_status",
            "ced_assignment_type_code" ,
            "item_type_code_name",
            "learner_attempt_status",
            "response_correctness",
            "scoring_type_code" ).show(5,False)

+-------+--------------------+------------------------+-------------------+----------------------+--------------------+-----------------+
summary|assigned_item_status|ced_assignment_type_code|item_type_code_name|learner_attempt_status|response_correctness|scoring_type_code|
+-------+--------------------+------------------------+-------------------+----------------------+--------------------+-----------------+
count |149807 |149807 |92062 |149807 |88760 |149807 |
mean |null |null |null |null |null |null |
stddev |null |null |null |null |null |null |
min |assigned |assessment |FillinBlankResponse|assigned |[unassigned] |[unassigned] |
max |scored |rubric |trueFalse |in progress |partially_correct |manual |
+-------+--------------------+------------------------+-------------------+----------------------+--------------------+-----------------+

In [8]:
# Display number of records
sqlContext.sql("SELECT is_deleted FROM raw_data").show(1)

+----------+
is_deleted|
+----------+
 false|
+----------+
only showing top 1 row

In [9]:
# Schema
dfRaw.printSchema()

root
-- org_sk: string (nullable = true)
-- section_sk: string (nullable = true)
-- ced_assignment_type_code: string (nullable = true)
-- is_affecting_grade: boolean (nullable = true)
-- assessment_sk: string (nullable = true)
-- assessment_instance_sk: string (nullable = true)
-- number_of_learners: integer (nullable = true)
-- number_of_distinct_instance_items: integer (nullable = true)
-- assignment_max_attempts: integer (nullable = true)
-- assignment_late_submission: boolean (nullable = true)
-- assignment_final_submission_date: string (nullable = true)
-- assignment_start_date: string (nullable = true)
-- assignment_due_date: string (nullable = true)
-- min_student_start_datetime: string (nullable = true)
-- max_student_stop_datetime: string (nullable = true)
-- assignment_attempt_number: integer (nullable = true)
-- learner_assignment_attempt_sk: string (nullable = true)
-- was_fully_scored_datetime: string (nullable = true)
-- was_submitted_datetime_actual: string (nullable = true)
-- was_in_progress_datetime: string (nullable = true)
-- is_force_scored: boolean (nullable = true)
-- is_manual_scoring_required: boolean (nullable = true)
-- student_start_datetime: string (nullable = true)
-- student_stop_datetime: string (nullable = true)
-- item_is_offline_scored: boolean (nullable = true)
-- assessment_instance_attempt_sk: string (nullable = true)
-- learner_attempt_status: string (nullable = true)
-- learner_sk: string (nullable = true)
-- points_possible_unweighted: double (nullable = true)
-- final_score_unweighted: integer (nullable = true)
-- scored_datetime: string (nullable = true)
-- learner_assigned_item_attempt_sk: string (nullable = true)
-- assessment_item_response_sk: string (nullable = true)
-- item_type_code_name: string (nullable = true)
-- scoring_type_code: string (nullable = true)
-- response_correctness: string (nullable = true)
-- assigned_item_status: string (nullable = true)
-- is_deleted: boolean (nullable = true)

In [10]:
# Identifier Data Summary
from pyspark.sql.functions import countDistinct
from functools import partial
from pyspark.sql import Row

# Convert columns to rows
def flatten_table(column_names, column_values):
    row = zip(column_names, column_values)
    _, key = next(row)  # Special casing retrieving the first column
    return [
        Row(Variable=column, Observations=value)
        for column, value in row
    ]

# Get the observation count with each type of record
counts = dfRaw.agg( 
  countDistinct("assessment_sk").alias("assessment_sk"), 
  countDistinct("assessment_instance_sk").alias("assessment_instance_sk"), 
  countDistinct("learner_assignment_attempt_sk").alias("learner_assignment_attempt_sk"), 
  countDistinct("assessment_instance_attempt_sk").alias("assessment_instance_attempt_sk"),  
  countDistinct("learner_assigned_item_attempt_sk").alias("learner_assigned_item_attempt_sk"), 
  countDistinct("assessment_item_response_sk").alias("assessment_item_response_sk"), 
  countDistinct("learner_sk").alias("learner_sk"), 
  countDistinct("section_sk").alias("section_sk"), 
  countDistinct("org_sk").alias("org_sk")
)
# Display the counts
counts.rdd.flatMap(partial(flatten_table, counts.columns)).toDF().sort('Variable').show(9, False)

+--------------------------------+------------+
Variable |Observations|
+--------------------------------+------------+
assessment_instance_attempt_sk |11780 |
assessment_instance_sk |824 |
assessment_item_response_sk |77746 |
learner_assigned_item_attempt_sk|147650 |
learner_assignment_attempt_sk |19013 |
learner_sk |1223 |
org_sk |3 |
section_sk |50 |
+--------------------------------+------------+

In [11]:
# Deleted records count
sqlContext.sql("SELECT count(distinct is_deleted) FROM raw_data WHERE is_deleted = true").show()


+--------------------------+
count(DISTINCT is_deleted)|
+--------------------------+
 0|
+--------------------------+

In [12]:
dfRaw.summary()

Out[18]: DataFrame[summary: string, org_sk: string, section_sk: string, ced_assignment_type_code: string, assessment_sk: string, assessment_instance_sk: string, number_of_learners: string, number_of_distinct_instance_items: string, assignment_max_attempts: string, assignment_final_submission_date: string, assignment_start_date: string, assignment_due_date: string, min_student_start_datetime: string, max_student_stop_datetime: string, assignment_attempt_number: string, learner_assignment_attempt_sk: string, was_fully_scored_datetime: string, was_submitted_datetime_actual: string, was_in_progress_datetime: string, student_start_datetime: string, student_stop_datetime: string, assessment_instance_attempt_sk: string, learner_attempt_status: string, learner_sk: string, points_possible_unweighted: string, final_score_unweighted: string, scored_datetime: string, learner_assigned_item_attempt_sk: string, assessment_item_response_sk: string, item_type_code_name: string, scoring_type_code: string, response_correctness: string, assigned_item_status: string]

In [13]:
%sql
SELECT COUNT(*) FROM raw_data where is_deleted;

count(1)
0
